In [1]:
#Setup

# Install Packages
import sys
!{sys.executable} -m pip install numpy --quiet
!{sys.executable} -m pip install matplotlib --quiet
!{sys.executable} -m pip install seaborn --quiet
!{sys.executable} -m pip install sklearn --quiet

from sklearn import preprocessing
import os, requests
from matplotlib import rcParams 
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import math
from sklearn.linear_model import LinearRegression
    
# Data Retrieval

fname = 'joystick_track.npz'
url = "https://osf.io/6jncm/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)
# Import matplotlib and set styling
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

colourmap_diverge = sns.diverging_palette(321, 172, s=100, n=100, center = "light", as_cmap=True)
colourmap = sns.color_palette("rocket", as_cmap=True)
colourmap = sns.light_palette("#30887c", as_cmap=True)
colourmap_diverge.set_bad("black", alpha=0)
colourmap.set_bad("black", alpha=0)

# Data Loading
alldat = np.load(fname, allow_pickle=True)['dat']

# Select just one of the recordings here. This is subject 1, block 1.
dat = alldat[0, 3]

You should consider upgrading via the 'C:\Users\richa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\richa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\richa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\richa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [47]:
# Load patient 2
dat = alldat[0, 2]
# Selected channels of high correlations for patient 2
corr_chan = sorted([5, 23, 17, 1, 25])

V = dat['V']
V_corr = dat['V'][:,corr_chan]
nt, nchan = V_corr.shape
print(nt, nchan)

cx = dat['cursorX'].flatten()
cy = dat['cursorY'].flatten()

134360 5


In [48]:
# Number of hindsight values
hindsight = 2
# lag between hindight values
lag = 500

# Generate feature indices
i0 = h*l
iN = nt
feature_step = 200
indices = range(i0, iN, feature_step)

# Generate 
no_obs = len(indices)
no_vars_rgand = 2    #Number of regressand variables (x and y)
no_vars_rgor = len(corr_chan) * (h+1)

regressand = np.zeros((no_obs, no_vars_rgand))
regressor = np.zeros((no_obs, no_vars_rgor))

for j, ind in enumerate(indices):
    regressand[j, 0] = cx[ind]
    regressand[j, 1] = cy[ind]
    for i, chan in enumerate(corr_chan):
        for h in range(hindsight+1):
            regressor[j, (hindsight+1)*i + h] = V_corr[ind - h * lag, i]
            print(regressor[j, (hindsight+1)*i + h])


print(columns)

#TO DO test and train splits





1.1279296875
1.128
 
0.150390625
0.1504
 
-1.4189453125
-1.419
 
-0.41552734375
-0.4155
 
-0.6796875
-0.6797
 
-0.08526611328125
-0.08527
 
1.8583984375
1.858
 
0.28125
0.2812
 
-3.1484375
-3.148
 
-0.8525390625
-0.8525
 
0.30712890625
0.3071
 
-0.94677734375
-0.947
 
0.9931640625
0.993
 
-0.060211181640625
-0.0602
 
-2.166015625
-2.166
 
1.9951171875
1.995
 
1.9931640625
1.993
 
0.7490234375
0.749
 
-0.7861328125
-0.786
 
1.20703125
1.207
 
0.51025390625
0.5103
 
2.447265625
2.447
 
1.5869140625
1.587
 
-0.34326171875
-0.3433
 
-0.513671875
-0.5137
 
0.6650390625
0.665
 
0.45458984375
0.4546
 
0.919921875
0.92
 
0.76416015625
0.764
 
-0.1640625
-0.1641
 
2.220703125
2.22
 
1.44921875
1.449
 
0.491943359375
0.492
 
-0.0699462890625
-0.06995
 
0.8232421875
0.823
 
-0.06591796875
-0.0659
 
2.63671875
2.637
 
1.720703125
1.721
 
0.324951171875
0.325
 
0.1622314453125
0.1622
 
-0.896484375
-0.8965
 
1.2705078125
1.2705
 
2.8984375
2.898
 
1.0302734375
1.03
 
0.09539794921875
0.0954
 
1.526

-0.2049560546875
-0.205
 
0.9384765625
0.9385
 
-1.27734375
-1.277
 
1.4609375
1.461
 
0.8505859375
0.8506
 
-0.32080078125
-0.3208
 
1.4765625
1.477
 
0.7890625
0.789
 
-1.642578125
-1.643
 
0.7939453125
0.794
 
0.6279296875
0.628
 
-0.64306640625
-0.643
 
0.6630859375
0.663
 
-0.479248046875
-0.4792
 
1.15625
1.156
 
-0.66552734375
-0.6655
 
-0.133056640625
-0.133
 
0.8583984375
0.8584
 
1.3896484375
1.39
 
0.06768798828125
0.0677
 
0.9208984375
0.921
 
-0.016448974609375
-0.01645
 
0.30859375
0.3086
 
0.35546875
0.3555
 
0.98095703125
0.981
 
-0.09130859375
-0.0913
 
0.68408203125
0.684
 
-1.1279296875
-1.128
 
1.50390625
1.504
 
0.73828125
0.7383
 
-2.123046875
-2.123
 
0.65087890625
0.651
 
0.828125
0.828
 
0.7275390625
0.7275
 
1.48046875
1.48
 
0.7119140625
0.712
 
-1.8310546875
-1.831
 
1.1552734375
1.155
 
0.07452392578125
0.0745
 
0.77490234375
0.775
 
0.96630859375
0.9663
 
0.41796875
0.418
 
0.1920166015625
0.192
 
0.935546875
0.9355
 
0.2724609375
0.2725
 
-0.515625
-0.515

-0.321044921875
-0.321
 
-1.0712890625
-1.071
 
0.255859375
0.2559
 
-0.607421875
-0.6074
 
-0.42333984375
-0.4233
 
-0.384765625
-0.3848
 
-1.3046875
-1.305
 
-0.2105712890625
-0.2106
 
0.30078125
0.3008
 
-0.7646484375
-0.7646
 
0.197998046875
0.198
 
-0.386962890625
-0.387
 
-0.8994140625
-0.8994
 
-0.474365234375
-0.4744
 
0.0989990234375
0.099
 
-1.2568359375
-1.257
 
0.76904296875
0.769
 
-0.1947021484375
-0.1947
 
-0.287841796875
-0.2878
 
-0.085693359375
-0.0857
 
0.23779296875
0.2378
 
-2.115234375
-2.115
 
-0.8134765625
-0.8135
 
1.197265625
1.197
 
-1.158203125
-1.158
 
0.449951171875
0.45
 
0.15283203125
0.1528
 
-1.48046875
-1.48
 
-0.77978515625
-0.78
 
0.96484375
0.965
 
-0.234619140625
-0.2346
 
0.396484375
0.3965
 
0.56787109375
0.568
 
-1.2109375
-1.211
 
-0.445068359375
-0.445
 
-0.26220703125
-0.2622
 
-0.3310546875
-0.331
 
-0.60546875
-0.6055
 
0.1300048828125
0.13
 
0.1180419921875
0.11804
 
0.32666015625
0.3267
 
-0.1593017578125
-0.1593
 
-0.1339111328125
-0.13

1.133
 
0.1595458984375
0.1595
 
0.7236328125
0.7236
 
0.2091064453125
0.2091
 
-2.119140625
-2.12
 
0.34228515625
0.3423
 
0.78125
0.7812
 
-0.7783203125
-0.7783
 
0.39306640625
0.393
 
1.0361328125
1.036
 
1.244140625
1.244
 
0.580078125
0.58
 
0.7412109375
0.741
 
-1.005859375
-1.006
 
-0.1485595703125
-0.1486
 
0.84228515625
0.8423
 
1.24609375
1.246
 
-0.0204315185546875
-0.02043
 
0.5078125
0.508
 
-1.4326171875
-1.433
 
-0.3232421875
-0.3232
 
0.329833984375
0.3298
 
1.0546875
1.055
 
0.2196044921875
0.2196
 
-0.1842041015625
-0.1842
 
1.4189453125
1.419
 
1.0048828125
1.005
 
-1.1201171875
-1.12
 
0.60546875
0.6055
 
-0.61962890625
-0.6196
 
-0.31982421875
-0.3198
 
1.17578125
1.176
 
0.57568359375
0.5757
 
-0.057403564453125
-0.0574
 
0.201171875
0.2012
 
-1.1513671875
-1.151
 
-0.307373046875
-0.3074
 
-0.751953125
-0.752
 
-1.888671875
-1.889
 
-0.1458740234375
-0.1459
 
0.65380859375
0.654
 
0.350341796875
0.3503
 
-0.34130859375
-0.3413
 
-0.357666015625
-0.3577
 
-1.46582

 
0.6943359375
0.6943
 
0.7001953125
0.7
 
0.05145263671875
0.05145
 
0.552734375
0.5527
 
0.358154296875
0.3582
 
-0.347900390625
-0.348
 
1.228515625
1.229
 
1.029296875
1.029
 
0.1632080078125
0.1632
 
-0.140869140625
-0.1409
 
1.033203125
1.033
 
0.28369140625
0.2837
 
-0.4892578125
-0.4893
 
1.060546875
1.061
 
0.11383056640625
0.11383
 
-0.1058349609375
-0.10583
 
0.76611328125
0.766
 
0.1920166015625
0.192
 
-1.408203125
-1.408
 
0.95263671875
0.9526
 
-0.51953125
-0.5195
 
0.59423828125
0.594
 
0.9072265625
0.907
 
0.341552734375
0.3416
 
0.399169921875
0.3992
 
0.59619140625
0.596
 
1.7412109375
1.741
 
-0.64013671875
-0.64
 
1.2880859375
1.288
 
1.109375
1.109
 
-0.10540771484375
-0.1054
 
0.94384765625
0.944
 
1.041015625
1.041
 
-0.58056640625
-0.5806
 
0.88720703125
0.887
 
0.794921875
0.795
 
0.5546875
0.5547
 
1.314453125
1.314
 
1.2978515625
1.298
 
-0.4716796875
-0.4717
 
0.1689453125
0.169
 
0.324951171875
0.325
 
-0.7197265625
-0.7197
 
0.53173828125
0.5317
 
0.57275

 
1.4560546875
1.456
 
-1.373046875
-1.373
 
-0.491455078125
-0.4915
 
0.385009765625
0.385
 
-1.66796875
-1.668
 
-0.56103515625
-0.561
 
-0.450927734375
-0.451
 
1.1787109375
1.179
 
-1.451171875
-1.451
 
0.49267578125
0.4927
 
1.21875
1.219
 
-1.646484375
-1.646
 
-0.239013671875
-0.239
 
0.2496337890625
0.2496
 
-1.5048828125
-1.505
 
-0.423828125
-0.4238
 
0.314697265625
0.3147
 
-1.103515625
-1.104
 
0.415283203125
0.4153
 
0.373291015625
0.3733
 
-1.4619140625
-1.462
 
0.01476287841796875
0.01476
 
1.2314453125
1.231
 
-1.4248046875
-1.425
 
0.74462890625
0.7446
 
0.97216796875
0.972
 
-1.3154296875
-1.315
 
-0.46337890625
-0.4634
 
0.311767578125
0.3118
 
-1.51171875
-1.512
 
1.1552734375
1.155
 
1.9833984375
1.983
 
-0.2459716796875
-0.246
 
-0.18701171875
-0.187
 
0.70703125
0.707
 
-1.638671875
-1.639
 
-0.9677734375
-0.968
 
-0.113037109375
-0.11304
 
-1.248046875
-1.248
 
-0.5078125
-0.508
 
0.465087890625
0.465
 
-1.1259765625
-1.126
 
-1.87890625
-1.879
 
-0.281982421875

-0.5176
 
0.1705322265625
0.1705
 
-0.051116943359375
-0.05112
 
0.0274200439453125
0.02742
 
-0.5517578125
-0.552
 
0.27294921875
0.273
 
0.260498046875
0.2605
 
0.1563720703125
0.1564
 
0.18115234375
0.1812
 
-0.640625
-0.6406
 
-0.57080078125
-0.571
 
0.481201171875
0.4812
 
-0.1878662109375
-0.1879
 
0.1319580078125
0.132
 
0.315185546875
0.3152
 
1.244140625
1.244
 
-0.012542724609375
-0.01254
 
0.43603515625
0.436
 
1.3916015625
1.392
 
-0.320068359375
-0.32
 
0.2437744140625
0.2438
 
1.546875
1.547
 
0.4521484375
0.4521
 
0.332275390625
0.3323
 
0.6279296875
0.628
 
-0.94677734375
-0.947
 
0.46044921875
0.4604
 
1.158203125
1.158
 
0.26904296875
0.269
 
0.45556640625
0.4556
 
2.103515625
2.104
 
0.19970703125
0.1997
 
-0.302490234375
-0.3025
 
1.83984375
1.84
 
0.23974609375
0.2397
 
-1.3916015625
-1.392
 
2.408203125
2.408
 
0.32861328125
0.3286
 
0.04766845703125
0.04767
 
1.5458984375
1.546
 
-0.76220703125
-0.762
 
-1.6494140625
-1.649
 
2.109375
2.11
 
-0.096923828125
-0.09

 
2.08203125
2.082
 
-0.1383056640625
-0.1383
 
1.2998046875
1.3
 
0.4892578125
0.4893
 
-1.783203125
-1.783
 
1.3076171875
1.308
 
2.353515625
2.354
 
-0.4404296875
-0.4404
 
0.39599609375
0.396
 
0.6123046875
0.6123
 
0.07769775390625
0.0777
 
-0.0260009765625
-0.026
 
-0.273193359375
-0.2732
 
-0.76904296875
-0.769
 
1.2392578125
1.239
 
0.9677734375
0.968
 
1.0810546875
1.081
 
-0.498291015625
-0.4983
 
-0.58056640625
-0.5806
 
-1.2783203125
-1.278
 
0.927734375
0.9277
 
0.9375
0.9375
 
0.75537109375
0.7554
 
-0.25927734375
-0.2593
 
0.78466796875
0.7847
 
1.3583984375
1.358
 
-1.2333984375
-1.233
 
0.126708984375
0.1267
 
0.305908203125
0.306
 
0.400634765625
0.4006
 
1.8564453125
1.856
 
1.9912109375
1.991
 
-0.59228515625
-0.5923
 
0.9716796875
0.9717
 
-0.78466796875
-0.7847
 
0.16552734375
0.1655
 
1.57421875
1.574
 
2.060546875
2.06
 
-0.732421875
-0.7324
 
-0.1876220703125
-0.1876
 
2.150390625
2.15
 
-1.2919921875
-1.292
 
-0.314208984375
-0.3142
 
0.6884765625
0.6885
 
-0.

 
0.64306640625
0.643
 
0.342529296875
0.3425
 
1.224609375
1.225
 
0.8671875
0.867
 
0.0535888671875
0.0536
 
-0.79931640625
-0.7993
 
-0.010772705078125
-0.01077
 
0.7421875
0.742
 
0.369140625
0.3691
 
0.603515625
0.6035
 
0.35498046875
0.355
 
1.9599609375
1.96
 
2.181640625
2.182
 
0.69189453125
0.692
 
-0.357421875
-0.3574
 
0.320068359375
0.32
 
0.1219482421875
0.12195
 
1.6611328125
1.661
 
1.451171875
1.451
 
0.90185546875
0.902
 
-1.033203125
-1.033
 
0.033966064453125
0.03397
 
0.276611328125
0.2766
 
0.1295166015625
0.1295
 
0.7919921875
0.792
 
-0.40771484375
-0.4077
 
2.42578125
2.426
 
1.603515625
1.604
 
-0.09552001953125
-0.0955
 
0.0484619140625
0.04846
 
-0.181396484375
-0.1814
 
-0.306884765625
-0.307
 
1.3232421875
1.323
 
1.380859375
1.381
 
0.474609375
0.4746
 
-0.487548828125
-0.4875
 
-0.435302734375
-0.4353
 
-0.64404296875
-0.644
 
-1.1318359375
-1.132
 
-0.6884765625
-0.6885
 
1.0244140625
1.024
 
-0.00893402099609375
-0.008934
 
1.076171875
1.076
 
1.977539

In [52]:
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3


reg = LinearRegression().fit(regressor, regressand)
print(reg.score(regressor, regressand))

print(reg.coef_)

print(reg.intercept_)

#reg.predict(np.array([[3, 5]]))


0.4369926286146171
[[ -934.79391546 -1176.84928427  -838.86906779 -2459.01108386
   -627.70737537 -1555.54998671  3008.66841062   805.87024224
   2211.00172915 -1739.67839872 -1394.39052664  -568.99307624
   1724.08832843   668.68771358   780.78570546]
 [   23.52031767   -30.17741006 -1231.20297702 -1475.33430665
  -2123.06231325 -3170.207189   -5054.10600277    25.07214161
      8.73256883  1011.09141521  -158.74964363   264.69127397
   2261.49588396   943.25088396  2373.09282773]]
[21141.07073153 22029.23149272]
